# Implementing a Voting System

## Background Theory
Here I will summarize the formalism used/developed in Wayland2019. The construction of a generalized voting system, consisting of a set of voters, a set of candidates and a voting rule to compute the outcome of an election is quite standard in the field of social choice theory.
* Let $V$ be a nonempty set of n voters $\{1,...,n\}$
* Let $C$ be the set of m candidates $\{c_1,...,c_m\}$
* In this situation, we assume that each voter is ranking each of the candidates on their ballot to produce a linear order.
* Let $\mathcal{L}(C)$ denote the set of all linear orders on $C$. It follows then that the collection of ballots, which will be referred to as a profile $\mathbb{P}$, is a subset of $\mathcal{L}(C)$. 
* For $i\in V$ let $P_i\in \mathcal{L}(C)$ denote the truthful preferences of the $i^{th}$ voter.
* Then a voting rule, $f$ is nothing more than a function $f:\mathcal{L}(C)\to C$ that picks out a single winner.

### Voting Rules
There are a number of different ways to determine the outcome of an election. This investigation will only consider positional scoring rules. These assign a numerical score to each candidate based on their rank in each $P_i$; the candidate with the maximum score is selected as the winner. 
* Define a scoring vector to be $s = \langle s_1,s_2,...,s_m \rangle$ where for $j=1,...,m-1, s_j\geq s_{j+1}$.
* For $x\in C$, let the function $score(P_i,x) = s_r$. This picks out the appropriate score in the scoring vector assocaited with the $r^{th}$ element in the linear order $P_i$.
* Thus in collecting the total score for $x\in C$ we are computing $total(\mathbb{P},x) = \sum_{i=1}^n score(P_i,x)$
* This allows for a concrete definition of a voting rule $f$ on a given profile $\mathbb{P}$ and scoring vector $s$:
    $$ f(\mathbb{P},s) = \max_{x\in C} total(\mathbb{P},x)$$
##### This formalism allows you to fully generate a postional scoring rule given it's scoring vector. Here are some specifications of a few common voting rules:
* Plurality: the rule given by $\langle 1,...,0 \rangle$.
* k-approval: the rule given by $\langle 1,1,...,0 \rangle$ where you have $k$ 1's followed by zeros.
* Borda: the rule given by $\langle m-1,m-2,...,0 \rangle$

## Implementation

In [88]:
# Imports

In [97]:
# Setup an example Election
C <- c("Donald Trump","Joe Biden")
num_voters <- 10
voting_rule <- "Plurality"

#Need to define function to generate the score vector for given number of candidates according to voting_rule

In [98]:
# Generate random ballots for the voters to construct a voting profile 

construct_profile <- function(num_voters,candidates) {    
    #' Return n random permutations on the set of canidates as a matrix
    profile <- matrix(data = seq(num_voters*length(candidates)),nrow = num_voters, ncol = length(candidates))
    for (i in seq(num_voters)) {
        ballot <- sample(C, length(C), replace=FALSE)
        profile[i,] <- ballot
    }
    return(profile)
}

In [99]:
# Compute the winner of the election 
f <- function(profile, s_vector) {
    
    #Want to be able to determine the outcome purely from the profile 
    #and the scoring rule as suggested in the theory section.
    #Want to find package (maybe use data frames) to get the count of each input in a column and combine with s_vector
    # to give total score
    # Then take the maximum and return winning candidate (might need to institute tie breaking rules at some point)
    
    
}

In [80]:
P <- construct_profile(num_voters,C)
P

Joe Biden,Donald Trump
Donald Trump,Joe Biden
Donald Trump,Joe Biden
Donald Trump,Joe Biden
Joe Biden,Donald Trump
Joe Biden,Donald Trump
Joe Biden,Donald Trump
Donald Trump,Joe Biden
Joe Biden,Donald Trump
Donald Trump,Joe Biden
